In [19]:
import time
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 定义参数
model_checkpoint = "Helsinki-NLP/opus-mt-zh-en"
checkpoint_path = "./saves/step_86500_bleu_29.87.bin"  # 假设使用训练中的checkpoint

# 加载tokenizer和模型
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# 加载checkpoint
#model.load_state_dict(torch.load(checkpoint_path, map_location='cpu')["model_state_dict"])
model.eval()

# 将模型转移到设备
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [10]:
def infer_translation_batch(input_texts, model, tokenizer, max_length=512, num_beams=1, length_penalty=1):
    # 记录推理开始时间
    start_time = time.time()

    # 预处理输入文本（批量处理）
    inputs = tokenizer(
        input_texts,
        return_tensors="pt",
        padding=True,  # 使用动态填充，对齐批量输入的长度
        truncation=True,
        max_length=max_length,
    ).to(device)

    # 模型生成翻译
    with torch.no_grad():
        output_tokens = model.generate(
            inputs["input_ids"],
            num_beams=num_beams,
            length_penalty=length_penalty,
            early_stopping=False,
            #temperature=0.5,
            #top_p=0.90,
            do_sample=False
        )

    # 解码生成的tokens为文本（批量处理）
    translations = [
        tokenizer.decode(output, skip_special_tokens=True) for output in output_tokens
    ]

    # 记录推理结束时间
    end_time = time.time()
    inference_time = end_time - start_time

    return translations, inference_time

def translate(input_text, model, tokenizer, batch_size=16):
    lines = input_text.splitlines()
    
    # 存储每一行的翻译结果
    translations = []
    total_time = 0
    
    # 分批处理
    for i in range(0, len(lines), batch_size):
        batch_lines = [line for line in lines[i:i + batch_size] if line.strip()]
        if not batch_lines:
            translations.extend([""] * len(batch_lines))
            continue
        batch_translations, time_cost = infer_translation_batch(batch_lines, model, tokenizer)
        translations.extend(batch_translations)
        total_time += time_cost
    
    final_translation = "\n".join(translations)
    
    return final_translation, total_time


In [20]:
# 用户输入
input_text = '''
为了降低Transformer翻译模型（如基于Helsinki-NLP的Opus模型）的推理时间并提高性能，以下是一些常见且有效的优化方法：

1. 模型量化
简介：量化是通过使用低精度数值表示模型权重（例如将32位浮点数转换为8位整数）来减少模型的计算量和内存占用，从而加快推理速度。
方法：
Post-training quantization (PTQ)：模型训练后对权重进行量化。
Quantization-aware training (QAT)：在训练时引入量化，通常效果比PTQ更好。
2. 模型剪枝
简介：剪枝通过移除模型中对推理结果影响较小的权重和节点来减小模型规模，从而加速推理。
方法：
结构化剪枝：移除整个层、注意力头或神经元。
非结构化剪枝：移除个别的低权重参数。
3. 减少模型尺寸
简介：通过使用更小的模型架构（例如减少层数、隐藏层维度或注意力头的数量），可以减少计算量和推理时间。
方法：使用较小版本的模型，例如opus-mt-small，或手动调整Transformer的超参数。
4. 启用混合精度推理
简介：混合精度推理允许部分计算使用半精度浮点数（FP16），从而减少内存占用并提高推理速度。
工具：
NVIDIA的TensorRT和**AMP (Automatic Mixed Precision)**是常用的工具，可以自动处理FP16的计算。
5. 使用高效的解码策略
简介：解码策略的选择影响推理速度。常用的解码方式如Beam Search虽然精度较高，但速度较慢。
方法：
降低beam size：减小beam size可以显著加快解码速度，虽然可能会略微牺牲翻译质量。
Top-k sampling和Nucleus Sampling (Top-p sampling)：这些方法通过限制词汇选择的范围来加快推理速度。
'''

# 进行推理并测量时间
translated_text, time_taken = translate(input_text, model, tokenizer)

# 输出结果
print(f"Original Text: \n{input_text}\n\n")
print(f"Translated Text: \n{translated_text}\n")
print(f"Inference Time: {time_taken:.4f} seconds")


Original Text: 

为了降低Transformer翻译模型（如基于Helsinki-NLP的Opus模型）的推理时间并提高性能，以下是一些常见且有效的优化方法：

1. 模型量化
简介：量化是通过使用低精度数值表示模型权重（例如将32位浮点数转换为8位整数）来减少模型的计算量和内存占用，从而加快推理速度。
方法：
Post-training quantization (PTQ)：模型训练后对权重进行量化。
Quantization-aware training (QAT)：在训练时引入量化，通常效果比PTQ更好。
2. 模型剪枝
简介：剪枝通过移除模型中对推理结果影响较小的权重和节点来减小模型规模，从而加速推理。
方法：
结构化剪枝：移除整个层、注意力头或神经元。
非结构化剪枝：移除个别的低权重参数。
3. 减少模型尺寸
简介：通过使用更小的模型架构（例如减少层数、隐藏层维度或注意力头的数量），可以减少计算量和推理时间。
方法：使用较小版本的模型，例如opus-mt-small，或手动调整Transformer的超参数。
4. 启用混合精度推理
简介：混合精度推理允许部分计算使用半精度浮点数（FP16），从而减少内存占用并提高推理速度。
工具：
NVIDIA的TensorRT和**AMP (Automatic Mixed Precision)**是常用的工具，可以自动处理FP16的计算。
5. 使用高效的解码策略
简介：解码策略的选择影响推理速度。常用的解码方式如Beam Search虽然精度较高，但速度较慢。
方法：
降低beam size：减小beam size可以显著加快解码速度，虽然可能会略微牺牲翻译质量。
Top-k sampling和Nucleus Sampling (Top-p sampling)：这些方法通过限制词汇选择的范围来加快推理速度。



Translated Text: 
To reduce the time of reasoning and improve performance of the Transformer translation model (e.g., the Opus model based on Helsinki-NLP), the following are common and effective m